In [2]:
import torch
import torchvision.transforms as T
from torchvision.models.segmentation import deeplabv3_resnet50
import cv2
import numpy as np 
from PIL import Image

In [3]:
image_cv2 = cv2.imread("123123.jpg")
orig_image = image_cv2.copy()
image_cv2 = cv2.cvtColor(image_cv2, cv2.COLOR_BGR2RGB)
image_PIL = Image.fromarray(image_cv2)

In [4]:
transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
image = transform(image_PIL).unsqueeze(0)


In [5]:
model = deeplabv3_resnet50(pretrained=True)
model.eval()

C:\Users\fluri\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\fluri\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [6]:
with torch.no_grad():
    output = model(image)['out'][0]
output_predictions = output.argmax(0)

In [7]:
mask = output_predictions == output_predictions.max()


In [8]:
mask = output_predictions.cpu().numpy()  # Move tensor to CPU and convert to NumPy array
mask = (mask == mask.max()).astype(np.uint8) * 255  # Create binary mask for the person
mask = cv2.resize(mask, (orig_image.shape[1], orig_image.shape[0]))


In [9]:
blurred_background = cv2.GaussianBlur(orig_image, (21, 21), 0)
mask_inv = cv2.bitwise_not(mask)

In [10]:
person = cv2.bitwise_and(orig_image, orig_image, mask=mask)
background = cv2.bitwise_and(blurred_background, blurred_background, mask=mask_inv)


In [11]:
final_image = cv2.add(person, background)
final_image_bgr = cv2.cvtColor(final_image, cv2.COLOR_RGB2BGR)

In [13]:
cv2.imwrite('blurred_background_image.jpg', final_image)

True